<a href="https://colab.research.google.com/github/Ishaqgopang/Agents_as_Tool/blob/master/text_to_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq diffusers transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from google.colab import userdata
HF_API_KEY = userdata.get('HF_API_KEY')

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype = torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

The TextToVideoSDPipeline has been deprecated and will not receive bug fixes or feature updates after Diffusers version 0.33.1. 


In [5]:


prompt = "a jungle boy playing in the jungle 30 seconds video"
video_frames = pipe(prompt, num_inference_steps=240).frames

video_frames_tensor = torch.stack([torch.from_numpy(np.array(frame)) for frame in video_frames])

print(f"Type of video_frames_tensor: {video_frames_tensor.dtype}")
print(f"Shape of video_frames_tensor: {video_frames_tensor.shape}")

if video_frames_tensor.ndim == 5 and video_frames_tensor.shape[0] == 1:
    video_frames_tensor = video_frames_tensor.squeeze(0)
    print(f"Shape after squeezing batch: {video_frames_tensor.shape}")

if video_frames_tensor.shape[-1] != 3: # Assuming 3 channels (RGB) should be the last dimension
    video_frames_tensor = video_frames_tensor.permute(0, 2, 3, 1)
    print(f"Shape after permute: {video_frames_tensor.shape}")


# Convert the PyTorch tensor back to a NumPy array
video_frames_np = video_frames_tensor.numpy()

video_path = export_to_video(video_frames_np)
video_name = video_path.replace('/temp', '')
print('Name:', video_name)
torch.cuda.empty_cache()

  0%|          | 0/240 [00:00<?, ?it/s]

Type of video_frames_tensor: torch.float32
Shape of video_frames_tensor: torch.Size([1, 16, 256, 256, 3])
Shape after squeezing batch: torch.Size([16, 256, 256, 3])
Name: /tmp/tmpnu7ws34n.mp4
